# Use Case Description

Automate key phases of incident investigation using a coordinated set of LLM-driven agents. These agents handle tasks such as triaging, summarizing, planning next steps, and generating recommendations, ultimately producing detailed investigation reports for SOC teams with minimal manual effort.

## Model used for this use case
This use case involves series of complex tasks so Reasoning Model would be more suitable.  However, the reasoning model has not been released yet so we will have to use the instruct model.

## Configuration
Update these variables to match your SageMaker deployment:

In [ ]:
# Update these variables to match your deployment
endpoint_name = 'foundation-sec-8b-endpoint'
aws_region = 'us-east-1'

print(f"Configuration:")
print(f"Endpoint: {endpoint_name}")
print(f"Region: {aws_region}")

In [ ]:
import boto3
import json
import re
import pandas as pd
from datetime import datetime
from IPython.display import display, Markdown

# Initialize SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=aws_region)

print(f"Connected to SageMaker endpoint: {endpoint_name}")

In [ ]:
# Generation arguments optimized for classification (short outputs)
generation_args = {
    "max_new_tokens": 10,  # Small since we only need class labels
    "temperature": None,   # Deterministic for consistent classification
    "repetition_penalty": 1.2,
    "do_sample": False,
    "use_cache": True,
}

print("Generation configuration for classification:")
for key, value in generation_args.items():
    print(f"  {key}: {value}")

In [ ]:
def invoke_endpoint(prompt, max_new_tokens=1024, temperature=0.3):
    """Invoke the SageMaker endpoint with the given prompt"""
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "do_sample": temperature > 0,
            "repetition_penalty": 1.2
        }
    }
    
    try:
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        result = json.loads(response['Body'].read().decode())
        
        # Handle different response formats from TGI
        if isinstance(result, list) and len(result) > 0:
            return result[0].get('generated_text', str(result))
        elif isinstance(result, dict):
            return result.get('generated_text', str(result))
        else:
            return str(result)
            
    except Exception as e:
        print(f"Error invoking endpoint: {str(e)}")
        return f"Error: {str(e)}"

# Test the endpoint connection
test_response = invoke_endpoint("Hello, can you help with cybersecurity?", max_new_tokens=50)
print("Test Response:")
print(test_response)

## Summarize Incident

In [5]:
def make_prompt_summarize_incident(metadata: dict, alert_logs: str) -> str:

    return (
        "You are a senior SOC analyst assisting with incident triage. "
        "Your task is to read the incident metadata and alert logs, and provide a clear summary of what occurred.\n\n"
        "Instructions:\n"
        "- Highlight the sequence of events (inferred from timestamps).\n"
        "- Think deeply about cause and effect and how artifacts relate to one another.\n"
        "- Mention key attack techniques used (if inferable from logs).\n"
        "- Describe how the attack began and progressed.\n"
        "- Use clear and concise language appropriate for L1/L2 analysts.\n\n"
        f"Incident Metadata:\n{metadata}\n\n"
        f"Alert Logs:\n{alert_logs}\n\n"
        "Summarize what happened in this incident in a few sentences"
    )


In [6]:
incident_meta = {
    "incident_id": "INC-1024",
    "type": "Unauthorized Access",
    "severity": "High",
    "timestamp": "2025-04-09T10:30:00Z"
}
raw_logs = """2025-04-09 10:00:23 - Alert: 5 failed login attempts for user 'alice' on host 'WS123'
2025-04-09 10:05:10 - Alert: Suspicious PowerShell execution on 'WS123' by 'alice' (malicious script blocked)
2025-04-09 10:10:45 - Alert: Process dumping LSASS memory on 'WS123' (possible credential theft)
2025-04-09 10:15:00 - Alert: Successful login of 'alice' to server 'DC1' from host 'WS123'"""

In [7]:
generated_summary = inference(make_prompt_summarize_incident(incident_meta, raw_logs))
display(Markdown(generated_summary))

**Incident Summary:** On April 9th at approximately 10:00 AM UTC, an unauthorized access incident occurred starting with five failed login attempts by user 'alice' on workstation WS123. This was followed by suspicious PowerShell activity around 10:05 AM, indicating potential malicious scripting which was blocked. At 10:10 AM, there was process dumping of LSASS memory, suggesting attempted credential harvesting. Finally, at 10:15 AM, 'alice' successfully logged into domain controller DC1 from WS123, likely using stolen credentials. The attack progression shows initial brute force attempts leading to privilege escalation via LSASS dump, culminating in lateral movement to a critical system. Key techniques include password spraying (failed logins), obfuscated files or information (PowerShell block), and OS credential dumping (LSASS). Severity is high due to successful unauthorized access and potential data compromise.

##  Identify Impacted Assets, Users, and MITRE Tactics

Identify impacted assets, users, and MITRE ATT&CK tactics/techniques from incident data. 

In [8]:
prompt = (
    "You are a cybersecurity expert specializing in threat analysis and incident response. Analyze the following incident logs and metadata, then:\n"
    "- List impacted host systems and IPs\n"
    "- List impacted user accounts\n"
    "- Identify MITRE ATT&CK tactics and techniques observed (with names or IDs)\n\n"
    f"Metadata: {incident_meta}\n"
    f"Logs:\n{raw_logs}\n\n"
    "Provide the result as a JSON object with no comments where"
    "- keys are 'impacted_hosts', 'impacted_users', 'tactics', all of which are lists"
    "- each tactic in 'tactics' should have 'name' and 'techniques' keys"
    "- each technique in 'techniques' should have 'id' and 'name' keys"
    "Only output JSON"
)

generated_entities = inference(prompt)

Cleanse the response data and read as json file

In [9]:
import json

match = re.search(r'\{[\s\S]*\}', generated_entities)
if match:
    try:
        generated_entities = json.loads(match.group(0))
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
else:
    print("No JSON object found in response.")
print(json.dumps(generated_entities, indent=2))

{
  "impacted_hosts": [
    "WS123",
    "DC1"
  ],
  "impacted_users": [
    "alice"
  ],
  "tactics": [
    {
      "name": "Credential Access",
      "techniques": [
        {
          "id": "T1003",
          "name": "OS Credential Dumping: LSASS Memory"
        }
      ]
    },
    {
      "name": "Execution",
      "techniques": [
        {
          "id": "T1059.001",
          "name": "Command and Scripting Interpreter: Windows Command Shell (PowerShell)"
        }
      ]
    },
    {
      "name": "Initial Access",
      "techniques": [
        {
          "id": "T1110",
          "name": "Brute Force: Password Guessing"
        }
      ]
    }
  ]
}


 ## Recommend Remediation Steps and Next Investigative Actions

 Recommend remediation steps and next investigative actions based on the incident summary and entities.

In [10]:
def make_prompt_for_recommend_actions(summary, entities) -> dict:
    impacted_hosts = ", ".join(entities.get("impacted_hosts", []))
    impacted_users = ", ".join(entities.get("impacted_users", []))
    tactics_list = [
        f"{tactic['name']}: {technique['name']} ({technique['id']})"
        for tactic in entities.get("tactics", [])
        for technique in tactic.get("techniques", [])
    ]

    context = (
        f"Incident Summary: {summary}\n"
        f"Impacted Hosts: {impacted_hosts}\n"
        f"Impacted Users: {impacted_users}\n"
        f"Observed Tactics: {tactics_list}\n\n"
    )

    return (
        "You are a SOC incident response assistant. Based on the incident details, respond strictly in the following JSON format:\n"
        '{\n  "remediation_steps": ["..."],\n  "next_steps": ["..."]\n}\n\n'
        + context
    )

In [11]:
_generated_recommend_actions = inference(make_prompt_for_recommend_actions(generated_summary, generated_entities))
match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", _generated_recommend_actions, re.DOTALL)
if match:
    generated_recommend_actions = json.loads(match.group(1))
    print(json.dumps(generated_recommend_actions, indent=2))
else:
    print("JSON was not correctly generated")

{
  "remediation_steps": [
    "Isolate WS123 and DC1 immediately from the network to prevent further lateral movement.",
    "Terminate any active sessions originating from WS123, especially those connected to DC1.",
    "Conduct a thorough forensic analysis on WS123 and DC1 to identify all compromised accounts and restore them from secure backups.",
    "Reset passwords for all users, particularly 'alice', enforcing strong, unique passwords and multi-factor authentication where possible.",
    "Revoke any existing session tokens and refresh Kerberos tickets across the domain.",
    "Patch WS123 and DC1 against known vulnerabilities related to LSASS exploitation.",
    "Monitor for additional signs of intrusion such as unusual file transfers, data exfiltration, or other malicious activities."
  ],
  "next_steps": [
    "Review and update firewall rules to restrict unnecessary access to domain controllers.",
    "Enhance logging and monitoring capabilities to detect similar brute-force

## Produce a Structured Incident Investigation Report
Compile a structured incident report using the summary, entities, and recommended actions.
Returns a formatted report as a text (could be markdown or plain text).

In [12]:
def make_report_prompt(metadata: dict, summary: str, entities: dict, actions: dict) -> str:
    """
    Compile a structured incident report using the summary, entities, and recommended actions.
    Returns a formatted report as a text (could be markdown or plain text).
    """
    hosts = ", ".join(entities.get("impacted_hosts", []))
    users = ", ".join(entities.get("impacted_users", []))
    tactics = [
        f"{tactic['name']}: {technique['name']} ({technique['id']})"
        for tactic in entities.get("tactics", [])
        for technique in tactic.get("techniques", [])
    ]
    remediation_list = "- " + "\n- ".join(
        step["text"] if isinstance(step, dict) and "text" in step else str(step)
        for step in actions.get("remediation_steps", [])
    )
    next_steps_list = "- " + "\n- ".join(
        step["text"] if isinstance(step, dict) and "text" in step else str(step)
        for step in actions.get("next_steps", [])
    )

    return (
        "You are a SOC analyst assistant tasked with writing the final incident report. "
        "Use the information below to fill out each section of the report.\n\n"
        f"Incident Metadata: {metadata}\n"
        f"Incident Summary: {summary}\n"
        f"Impacted Hosts: {hosts}\n"
        f"Impacted Users: {users}\n"
        f"Observed Tactics/Techniques: {tactics}\n"
        f"Remediation Steps Taken:\n{remediation_list}\n"
        f"Next Investigation Steps:\n{next_steps_list}\n\n"
        "Now format the incident report with the following sections:\n"
        "## Incident Summary\n"
        "*(A brief overview of the incident.)*\n"
        "## Impacted Assets and Users\n"
        "*(Which systems and accounts were affected.)*\n"
        "## Adversary Tactics and Techniques\n"
        "*(ATT&CK tactics/techniques observed in this incident.)*\n"
        "### Incident Details (Timeline)\n"
        "*(Chronological log of key events.)*\n\n"
        "## Remediation Actions Implemented\n"
        "*(How we contained/mitigated the incident.)*\n"
        "## Next Steps and Recommendations\n"
        "*(Further investigation steps or preventive measures.)*\n"
        "Draft the report now, incorporating all provided details in the appropriate sections."
        "Return in Markdown format"
    )


In [13]:
report = inference(make_report_prompt(incident_meta, generated_summary, generated_entities, generated_recommend_actions))
display(Markdown(report))

# Incident Report: Unauthorized Access - INC-1024

## Incident Summary
On April 9th, 2025, at approximately 10:00 AM UTC, an unauthorized access incident of high severity was detected. The sequence began with five consecutive failed login attempts by user 'alice' targeting workstation WS123. Shortly after, between 10:05 AM and 10:10 AM, suspicious PowerShell activity was identified, indicative of obfuscated scripts potentially used for malicious purposes. Following this, there was a process dump of Local Security Authority Subsystem Service (LSASS) memory, signifying an attempt to harvest credentials. By 10:15 AM, 'alice' managed to authenticate onto the domain controller DC1 from WS123, presumably utilizing obtained credentials. The attacker's methods included password spraying, leveraging obfuscated files through PowerShell, and executing OS credential dumping via LSASS. These actions led to successful unauthorized entry and pose significant risk for data breaches.

## Impacted Assets and Users
**Affected Systems:**
- Workstation: WS123
- Domain Controller: DC1

**Impacted User:**
- Alice

## Adversary Tactics and Techniques
The observed adversary tactics and techniques align with specific MITRE ATT&CK frameworks:

- **Initial Access**: Brute Force: Password Guessing (T1110)
- **Execution**: Command and Scripting Interpreter: Windows Command Shell (PowerShell) (T1059.001)
- **Credential Access**: OS Credential Dumping: LSASS Memory (T1003)

## Incident Details (Timeline)
- **10:00 AM**: Five failed login attempts by 'alice' on WS123.
- **10:05 AM - 10:10 AM**: Suspicious PowerShell activity detected; blocked due to obfuscation.
- **10:10 AM**: Process dumping of LSASS memory, indicating credential harvesting efforts.
- **10:15 AM**: Successful login of 'alice' to DC1 from WS123, implying use of stolen credentials.

## Remediation Actions Implemented
Immediate containment and mitigation steps were executed to halt the breach’s spread and minimize damage:

- Network isolation of WS123 and DC1 to prevent further lateral movement.
- Terminated active sessions originating from WS123, specifically those linked to DC1.
- Initiated comprehensive forensic analyses on both impacted hosts to recover and reset compromised accounts from secure backups.
- Forcibly changed passwords for all users, prioritizing 'alice,' enforced stronger policies including MFA wherever applicable.
- Revoked current session tokens and refreshed Kerberos tickets across the entire domain.
- Applied patches addressing known LSASS vulnerability exploits on WS123 and DC1.
- Enhanced ongoing surveillance for secondary indicators like data leaks or anomalous file movements.

## Next Steps and Recommendations
To fortify defenses and prevent recurrence:

- Update firewall configurations to limit non-essential accesses to domain controllers.
- Augment logging and detection tools to spot future brute-force assaults and credential extraction attempts promptly.
- Organize employee education sessions focused on recognizing phishing attempts and improving password management practices.
- Collaborate with IT to deploy advanced Endpoint Detection and Response (EDR) solutions offering superior threat identification and response mechanisms.
- Compile a detailed post-mortem report for executive briefing and archival, encapsulating responses and insights gleaned from this event.

---

This structured markdown document provides a clear, concise summary of the incident, its remediation, and recommended next steps.